In [2]:
# import sys
# sys.prefix == sys.base_prefix

import smtplib
import pandas as pd
import requests
from bs4 import BeautifulSoup
from price_parser import Price
import re

PRODUCT_URL_CSV = "products.csv"
SAVE_TO_CSV = True
PRICES_CSV = "prices.csv"
SEND_MAIL = True

def readUrls(csv_file):
    df = pd.read_csv(csv_file)    
    return df

def getResponse(url):
    headers = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
    response = requests.get(url,headers = headers)
    return response.text

def getPrice(html):

    soup = BeautifulSoup(html, "html.parser")
    #el ="₹1,21,999" #soup.find('span', {'class' :'a-price-whole'} ) #a-price-whole"  /a-offscreen
    #priceEl = soup.find("span", class_="a-offscreen")
    priceEl = soup.find("span", class_="a-price-whole")
    if priceEl:
        price = priceEl.get_text()
        priceNew = price.replace(",", "").replace("₹","")
        #print(priceNew)
    else:
        #print("The price element could not be found")
        priceNew = 0
    return float(priceNew)


def formatProducts(df):
    updated_products = []
    for product in df.to_dict("records"):
        html = getResponse(product["url"])
        product["price"] = getPrice(html) 
        print(product["price"]) #price check
        product["alert"] = product["price"] <= product["alertPrice"]
        updated_products.append(product)
    print(updated_products)
    return pd.DataFrame(updated_products)

def getMail(df):
    subject = "Price Drop Alert"
    body = df[df["alert"]].to_string()
    subject_and_message = f"Subject:{subject}\n\n{body}"
    return subject_and_message

def send_mail(df):
    message_text = getMail(df)
    with smtplib.SMTP("smtp.server.address", 587) as smtp:
        smtp.starttls()
        smtp.login(mail_user, mail_pass)
        smtp.sendmail(mail_user, mail_to, message_text)

def mainDriver():
    df = readUrls(PRODUCT_URL_CSV)
    display(df)
    df_updated = formatProducts(df)
#    if SAVE_TO_CSV:
#        df_updated.to_csv(PRICES_CSV, index=False, mode="a")
#        print("running")
    #if SEND_MAIL:
     #   send_mail(df_updated)

In [3]:
mainDriver()

,product,url,alertPrice
0,Nothing Phone(1),https://www.amazon.in/Nothing-Phone-Black-128-...,27950
1,iPhone 14,https://www.amazon.in/Apple-iPhone-128GB-Space...,121990


28850.0
0.0
[{'product': 'Nothing Phone(1)', 'url': 'https://www.amazon.in/Nothing-Phone-Black-128-RAM/dp/B0BKZVF7VV', 'alertPrice': 27950, 'price': 28850.0, 'alert': False}, {'product': 'iPhone 14', 'url': 'https://www.amazon.in/Apple-iPhone-128GB-Space-Black/dp/B0BDJ7P6NG', 'alertPrice': 121990, 'price': 0.0, 'alert': True}]
